# Projet

## Apercu du projet

Vous avez rejoint une nouvelle équipe dans le secteur de la banque de détail, qui connaît actuellement des taux de défaut plus élevés que prévu sur les prêts personnels. Les prêts personnels sont une source de revenus importante pour les banques, mais ils comportent le risque inhérent que les emprunteurs puissent faire défaut. Un défaut de paiement se produit lorsqu'un emprunteur cesse de faire les paiements requis sur une dette.

## Objectif : 

L'équipe de risque analyse le portefeuille de prêts existants pour prévoir les défauts potentiels futurs et estimer la perte attendue. L'objectif principal est de construire un modèle prédictif qui estime la probabilité de défaut pour chaque client en fonction de ses caractéristiques. Des prédictions précises permettront à la banque d'allouer suffisamment de capital pour couvrir les pertes potentielles, maintenant ainsi la stabilité financière.

### 1. Exploration du Dataset

In [22]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [23]:
# Chargement du fichier CSV
fichier = "Loan_Data.csv"
df = pd.read_csv(fichier)

In [24]:

df.head(10)

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0
5,4661159,0,5376.886873,7189.121298,85529.84591,2,697,0
6,8291909,1,3634.057471,7085.980095,68691.57707,6,722,0
7,4616950,4,3302.172238,13067.570210,50352.16821,3,545,1
8,3395789,0,2938.325123,1918.404472,53497.37754,4,676,0
9,4045948,0,5396.366774,5298.824524,92349.55399,2,447,0


In [25]:

print(df.dtypes)

customer_id                   int64
credit_lines_outstanding      int64
loan_amt_outstanding        float64
total_debt_outstanding      float64
income                      float64
years_employed                int64
fico_score                    int64
default                       int64
dtype: object


In [26]:
# Statistiques descriptives (sans 'customer_id')

print("\n Statistiques descriptives :")
df.describe().drop(columns=['customer_id'])


 Statistiques descriptives :


,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.461200,4159.677034,8718.916797,70039.901401,4.552800,637.557700,0.185100
std,1.743846,1421.399078,6627.164762,20072.214143,1.566862,60.657906,0.388398
min,0.000000,46.783973,31.652732,1000.000000,0.000000,408.000000,0.000000
25%,0.000000,3154.235371,4199.836020,56539.867903,3.000000,597.000000,0.000000
50%,1.000000,4052.377228,6732.407217,70085.826330,5.000000,638.000000,0.000000
75%,2.000000,5052.898103,11272.263740,83429.166133,6.000000,679.000000,0.000000
max,5.000000,10750.677810,43688.784100,148412.180500,10.000000,850.000000,1.000000


In [27]:
# Analyse des valeurs manquantes
print("\n Valeurs manquantes :")
print(df.isnull().sum())


 Valeurs manquantes :
customer_id                 0
credit_lines_outstanding    0
loan_amt_outstanding        0
total_debt_outstanding      0
income                      0
years_employed              0
fico_score                  0
default                     0
dtype: int64


In [28]:
# Vérifier s'il y a des doublons dans le DataFrame
nb_doublons = df.duplicated().sum()

if nb_doublons > 0:
    print(f"Il y a {nb_doublons} lignes dupliquées dans le dataset.")
else:
    print(" Aucun doublon détecté dans le dataset.")

 Aucun doublon détecté dans le dataset.


### 2. Pré-traitement

In [29]:
# Définition de la variable cible
target = "default"  # 🔁 adapte ici si ta colonne s'appelle autrement (ex: "loan_status")
X = df.drop(columns=[target])
y = df[target]

# Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 X_train: {X_train.shape}, X_test: {X_test.shape}")

# Normalisation (StandardScaler)
# On standardise pour centrer-réduire les features : (x - mean)/std
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Normalisation effectuée (StandardScaler appliqué sur X_train et X_test)")

📊 X_train: (8000, 7), X_test: (2000, 7)
Normalisation effectuée (StandardScaler appliqué sur X_train et X_test)


### 3. Test de 3 modèles de ML avec MLFLOW

In [30]:
import mlflow 

from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [32]:
# Connexion au tracking
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")